# Project Question

> Create a search function to search in a NAS space for the best performing **CNN architecture** on fashion-mnist.

Your task is to create a search function which searches for the model genome string that provides the best accuracy while using the least number of parameters.

Your program will return a model genome string (described below) which can be fed to a train function (provided below) to get the lowest parameter count and best accuracy after training.

**This is an open research problem**. You can use any method to search for the best genome and there might not be a perfect solution possible (prefect in the sense of maximum accuracy with minimum parameters). You will likely find tradeoffs between accuracy and number of parameters and you must balance such tradeoffs at your discretion.

> Aim for a search function that finds a genome with accuracy at least 75%.

You are encouraged to experiment with different techniques and apply your intuition to find a good search algorithm.

## CNN Architecture Constraints

Your search algorithm should search for models with the following basic structure.

Sequential model (no skip connections) having the following layers,
- Any number of normal(NC) CNN layers
  - `torch.nn.Conv2d` or `tf.keras.layers.Conv2D`
  - A normal CNN layer has the following constraints,
    - `stride=1`
    - `padding=same`
    - `1 <= kernel size < 8`
    - Any of the following activation functions,
      - relu
      - sigmoid
      - tanh
      - swish
      - gelu
- **Exactly** 2 reduction(RC) CNN layers
  - `torch.nn.Conv2d` or `tf.keras.layers.Conv2D`
  - A reduction layer has the **same** constraints as the normal layer except the following,
    - `stride=2`
    - `padding=valid`
- **Exactly** the following structure as the **final layers**,
  - First, a Global Average Pooling 2D layer.
    - `torch.nn.AvgPool2d(kernel_size=layer_input_image_size)` or `tf.keras.layers.GlobalAveragePooling2D`
  - Then, a Fully Connected/Dense layer with 64 units (such that output is shape `(batch_size, 10)`)
    - This layer can have any of the allowed activations in NC layer but all final layers must use same activation.
    - `torch.nn.Linear(64)` or `tf.keras.layers.Dense(64)`
  - Then, a Fully Connected/Dense layer with 10 units (such that output is shape `(batch_size, 10)`)
    - This layer can have any of the allowed activations in NC layer but all final layers must use same activation.
    - `torch.nn.Linear(10)` or `tf.keras.layers.Dense(10)`


The normal(NC) and reduction(RC) layers can be in **any order** as long as the constraints above are maintained.

The final layers of you model should be exactly as mentioned in these constraints.

All of the above contraints **apply only to your final output**. During searching for the best architecture you can do whatever you want.

Once your algorithm finds the best architecture, it should represent the architecture as a genome string (explained below) which will be fed to the training function given below to get the final accuracy.

In [1]:
#@title Run to setup question dependencies
!pip install --upgrade -q tfds-nightly tensorflow-addons

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [4]:
#@title Run to download and prepare dataset
#@markdown - https://www.tensorflow.org/datasets/catalog/fashion_mnist

import tensorflow as tf
import tensorflow_datasets as tfds

(__DS_TRAIN, __DS_TEST), __DS_INFO = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

_ = tfds.show_examples(__DS_TEST, __DS_INFO)

# Prepare splits
def __normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label
# Prepare train split
__DS_TRAIN = __DS_TRAIN.map(__normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
__DS_TRAIN = __DS_TRAIN.cache()
__DS_TRAIN = __DS_TRAIN.shuffle(__DS_INFO.splits['train'].num_examples)
__DS_TRAIN = __DS_TRAIN.batch(128)
__DS_TRAIN = __DS_TRAIN.prefetch(tf.data.experimental.AUTOTUNE)
# Prepare test split
__DS_TEST = __DS_TEST.map(__normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
__DS_TEST = __DS_TEST.batch(128)
__DS_TEST = __DS_TEST.cache()
__DS_TEST = __DS_TEST.prefetch(tf.data.experimental.AUTOTUNE)

OverflowError: Python int too large to convert to C long

In [9]:
#@title Run to setup model genome processing code
#@markdown A model genome is a string having `;` seperated parts where each part describes a specific layer.
#@markdown 
#@markdown For eg., this cell genome has 1 normal convolution layer followed by 2 reduction convolution layers and relu activation in the final layer.
#@markdown ```
#@markdown NC 10 2 sigmoid;RC 10 3 relu;RC 10 3 relu;FL relu;
#@markdown ```
#@markdown 
#@markdown In the above example the normal layer has 10 CNN filters having a kernel size of 2 and sigmoid activation. After the normal layer it has 2 reduction layers with 10 filters, kernel size of 3 and relu activation function. This example therefore has 3 internal layers. Finally, the genome specifies `relu` activation for the final layer(FL).
#@markdown 
#@markdown The format for each genome part for `NC` and `RC` layers is as follows,
#@markdown ```
#@markdown <NC or RC> <no. of CNN filters> <kernel size LESS THAN 8> <activation function>;
#@markdown ```
#@markdown After all `NC` and `RC` genome parts the final layer genome part has the following format,
#@markdown ```
#@markdown FL <activation function>;
#@markdown ```
#@markdown With the above, the full genome format is,
#@markdown ```
#@markdown <any number of FC/RC genome parts>;FL <activation function>;
#@markdown ```
#@markdown Here,
#@markdown - `NC` is normal genome and `RC` is reduction genome. Normal genomes keep the dimentions of the input image intact (i.e. `stride=1`), while reduction genomes make the output image dimension half of the input dimension.
#@markdown  - See the constraints section above for specific details.
#@markdown - Every genome **MUST** have **EXACTLY 2** `RC` parts. These `RC` parts can be placed anywhere in the genome.

import tensorflow as tf
import tensorflow_addons as tfa
import re
from collections import Counter

__ALLOWED_ACTIVATIONS = [
                         'relu',
                         'sigmoid',
                         'tanh',
                         'swish',
                         'gelu',
]
__VALIDATION_RE_CELL = re.compile(f'((?P<type>NC|RC) (?P<filters>\d+?) (?P<kernel>[1234567]) (?P<activation>{"|".join(__ALLOWED_ACTIVATIONS)});)+?')
__VALIDATION_RE_FL = re.compile(f'FL (?P<fl_activation>{"|".join(__ALLOWED_ACTIVATIONS)});')
__VALIDATION_RE = re.compile(f'((?P<type>NC|RC) (?P<filters>\d+?) (?P<kernel>[1234567]) (?P<activation>{"|".join(__ALLOWED_ACTIVATIONS)});)+?FL (?P<fl_activation>{"|".join(__ALLOWED_ACTIVATIONS)});')

__SAMPLE_GENOME = 'NC 1 1 sigmoid;RC 2 3 relu;RC 5 7 relu;'

def parse_genome(cellgenome: str):
  if __VALIDATION_RE.fullmatch(cellgenome) is None:
    raise Exception(f'Invalid genome (parse error): {cellgenome}\n Please check that the genome is in the correct format.')
  parse = __VALIDATION_RE_CELL.findall(cellgenome)
  # check constraints
  type_count = Counter()
  for idx, cell in enumerate(parse):
    type_count.update([cell[1]])
    filter = int(cell[2])
    kernel = int(cell[3])
    if filter < 1:
      raise Exception(f'Invalid filter size {filter} for layer {idx+1}.')
    if kernel < 1 or kernel > 7:
      raise Exception(f'Invalid kernel size {filter} for layer {idx+1}.')
  if type_count['RC'] != 2:
    raise Exception(f'Genome must have exacly 2 RC parts. Your genome has {type_count["RC"]} RC parts.')
  return parse + __VALIDATION_RE_FL.findall(cellgenome)

def genome_to_model(cellgenome: str, lr: float = 2e-3, lr_decay: float = 0.05):
  # parse genome
  parsed = parse_genome(cellgenome)
  def _match_to_layer(match_tuple, idx):
    return tf.keras.layers.Conv2D(
        filters = int(match_tuple[2]),
        kernel_size = int(match_tuple[3]),
        strides = 1 if match_tuple[1] == 'NC' else 2,
        name = f'Layer_{idx}_{match_tuple[1]}',
        activation = match_tuple[4],
        padding = 'same' if match_tuple[1] == 'NC' else 'valid',
    )
  # create model object
  model = tf.keras.Sequential(
    [
      *[_match_to_layer(cell, idx) for idx, cell in enumerate(parsed[:-1])],
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dense(
          64,
          activation=parsed[-1],
      ),
      tf.keras.layers.Dense(
          __DS_INFO.features['label'].num_classes,
          activation=parsed[-1],
      ),
    ]
  )
  # compile the model
  model.compile(
      optimizer=tfa.optimizers.AdamW(
          learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=lr,
            decay_steps=len(__DS_TRAIN)*10,
            decay_rate=lr_decay,
          ),
          weight_decay=1e-9,
      ),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy']
  )
  model.build((None, *__DS_INFO.features['image'].shape))
  return model

In [ ]:
#@title Use this form to validate a genome and see the resulting model
#@markdown Enter your genome in this text field and then run this cell.
__fr_model_genome = "RC 1 3 relu;NC 1 3 relu;RC 1 3 relu;NC 4 7 sigmoid;FL relu;" #@param {type:"string"}

try:
  __fr_model = genome_to_model(__fr_model_genome)
  __fr_model.summary()
  del __fr_model
except Exception as e:
  print(e)
  print()
  print(f'Your input genome: {__fr_model_genome}')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer_0_RC (Conv2D)         (None, 13, 13, 1)         10        
                                                                 
 Layer_1_NC (Conv2D)         (None, 13, 13, 1)         10        
                                                                 
 Layer_2_RC (Conv2D)         (None, 6, 6, 1)           10        
                                                                 
 global_average_pooling2d (G  (None, 1)                0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                128       
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                        

In [12]:
#@title Run to setup model training code
#@markdown You are provided with two functions,
#@markdown - **`get_model_parameters(model_genome: str)`**: This function accepts the model genome string and returns the number of parameters that the model will have. You can use this function to query for the number of parameters without training the model entirely.
#@markdown 
#@markdown    **Returns**: A single integer representing the number of parameters in the model.

#@markdown - **`train_model(model_genome: str, verbose: int = 1, lr: float = None, lr_decacy: float = None)`**: This function accepts the model genome string and trains the model on the Fashion-MNIST dataset. You must use this function to train models based on your genome string.
#@markdown 
#@markdown      - If `verbose` is `0` then this function does not print anything while training
#@markdown      - If `verbose` is `1` then it prints the training progess on each call and the final test accuracy.
#@markdown      - If `verbose` is `2` then it prints everything from `varbose=1` and also prints the model summary (structure).
#@markdown 
#@markdown  It accepts a learning rate (`lr`) and a learning rate exponential decay rate (`lr_decay`). You do not need to provide values for `lr` and `lr_decay`, the default values are tuned to give good performance  3~10 layer networks. Below you will find some example values for a sample shallow network and a sample deep network.
#@markdown 
#@markdown   **Returns**: A tuple of `(test accuracy, train accuracy, number of parameters)`.
#@markdown 
#@markdown Using the above two functions you must create a search method to search for a model genome which gives the maximum **test accuracy**. You are given the train accuracy to identify _overfitting_ if you wish to do so. You can also use the number of parameters to prefer models having fewer parameters and find a good balance between test accuracy and number of parameters.

def get_model_parameters(modelgenome: str):
  '''
  Returns the number of parameters that this genome creates.
  '''
  model = genome_to_model(modelgenome)
  return model.count_params()

def train_model(modelgenome: str, verbose: int = 1, lr: float = None, lr_decay: float = None):
  '''
  Accepts a genome string and trains the corresponding model on Fashion MNIST dataset.
  Thorws exceptions if the provided genome is invalid.
  Returns a tuple: (test accuracy, train accuracy, number of parameters)
  '''
  model = genome_to_model(modelgenome, lr=lr, lr_decay=lr_decay)
  n_params = model.count_params()
  if verbose > 0:
    print(f'Training genome: {modelgenome}')
    print(f'Model params: {n_params}')
  if verbose > 1:
    model.summary()
  # train model
  callbacks = [
    tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      mode='min',
      baseline=10.0,
      patience=2,
      min_delta=0.005,
      verbose=verbose,
    )
  ]
  if verbose > 0:
    callbacks.append(
      tfa.callbacks.TQDMProgressBar(
        update_per_second = 1,
        leave_epoch_progress = verbose > 1,
        leave_overall_progress = verbose > 1,
      )
    )
  ret = model.fit(
      __DS_TRAIN,
      epochs=10,
      verbose=0,
      callbacks=callbacks,
      validation_data=__DS_TEST,
  )
  # delete model
  del model
  # get best history
  best_idx = max(range(len(ret.history['val_accuracy'])), key=ret.history['val_accuracy'].__getitem__)
  best_history = {
      k: v[best_idx] for k, v in ret.history.items()
  }
  if verbose > 0:
    print(f'Best test accuracy: {best_history["val_accuracy"]}')
  return (best_history['val_accuracy'], best_history['accuracy'], n_params)

In [ ]:
#@title Use this form to train specific genomes
#@markdown Enter your genome in this text field and then run this cell.
#@markdown 
__fr_model_genome = "RC 64 4 relu;NC 64 5 relu;NC 128 5 relu;NC 128 4 sigmoid;RC 8 7 sigmoid;FL relu;" #@param {type:"string"}

#@markdown You can finetune training hyperparameters here, (given values are good defaults and you shouldn't have to change for good results)
__fr_lr = 1e-3 #@param {type:"number"}
__fr_lr_decay = 0.1 #@param {type:"number"}

try:
  # Train
  print('Return value:', train_model(__fr_model_genome, verbose=2, lr=__fr_lr, lr_decay=__fr_lr_decay))
except Exception as e:
  print(e)
  print()
  print(f'Your input genome: {__fr_model_genome}')

Training genome: RC 64 4 relu;NC 64 5 relu;NC 128 5 relu;NC 128 4 sigmoid;RC 8 7 sigmoid;FL relu;
Model params: 622162
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer_0_RC (Conv2D)         (None, 13, 13, 64)        1088      
                                                                 
 Layer_1_NC (Conv2D)         (None, 13, 13, 64)        102464    
                                                                 
 Layer_2_NC (Conv2D)         (None, 13, 13, 128)       204928    
                                                                 
 Layer_3_NC (Conv2D)         (None, 13, 13, 128)       262272    
                                                                 
 Layer_4_RC (Conv2D)         (None, 4, 4, 8)           50184     
                                                                 
 global_average_pooling2d_1   (None, 8)                0         
 

Training:   0%|           0/10 ETA: ?s,  ?epochs/s

Epoch 1/10


0/469           ETA: ?s - 

Epoch 2/10


0/469           ETA: ?s - 

Epoch 3/10


0/469           ETA: ?s - 

Epoch 00003: early stopping
Best test accuracy: 0.10000000149011612
Return value: (0.10000000149011612, 0.09830000251531601, 622162)


In [ ]:
#@title Deep model genome example
#@markdown Enter your genome in this text field and then run this cell.
#@markdown 
__fr_model_genome = "NC 16 6 relu;NC 16 5 sigmoid;NC 4 6 relu;RC 64 7 relu;RC 16 6 relu;FL relu;" #@param {type:"string"}

#@markdown You can finetune training hyperparameters here, (given values are good defaults and you shouldn't have to change for good results)
__fr_lr = 1e-3 #@param {type:"number"}
__fr_lr_decay = 0.1 #@param {type:"number"}

try:
  # Train
  print('Return value:', train_model(__fr_model_genome, verbose=2, lr=__fr_lr, lr_decay=__fr_lr_decay))
except Exception as e:
  print(e)
  print()
  print(f'Your input genome: {__fr_model_genome}')

Training genome: NC 16 6 relu;NC 16 5 sigmoid;NC 4 6 relu;RC 64 7 relu;RC 16 6 relu;FL relu;
Model params: 60542
Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer_0_NC (Conv2D)         (None, 28, 28, 16)        592       
                                                                 
 Layer_1_NC (Conv2D)         (None, 28, 28, 16)        6416      
                                                                 
 Layer_2_NC (Conv2D)         (None, 28, 28, 4)         2308      
                                                                 
 Layer_3_RC (Conv2D)         (None, 11, 11, 64)        12608     
                                                                 
 Layer_4_RC (Conv2D)         (None, 3, 3, 16)          36880     
                                                                 
 global_average_pooling2d_46  (None, 16)               0         
  (Glo

Training:   0%|           0/10 ETA: ?s,  ?epochs/s

Epoch 1/10


0/469           ETA: ?s - 

Epoch 2/10


...

Epoch 9/10


0/469           ETA: ?s - 

Epoch 10/10


0/469           ETA: ?s - 

Epoch 00010: early stopping
Best test accuracy: 0.4251999855041504
Return value: (0.4251999855041504, 0.42195001244544983, 60542)


## Hints

- Consider caching genome results for faster querying.
  - This has a risk. The training process itself is non-deterministic and thus training the same genome multiple times can sometimes give drastically different results. Generally such difference in accuracy of the same genome is small but sometimes it be upto 30%.
- Consider saving your search algorithm state such that it can resume from a previous search point if it gets interrupted due to a crash or colab time restrictions.
  - Google colab is free and hence it has time limits on how long you can use the GPU. If you implement checkpointing in your code such that it saves checkpoints on your GDrive, you can save a lot of time when such interruptions happen.
  - You can mount your GDrive to colab and save/load files from your drive folders.
- Don't forget to enable GPU in colab.
  - https://colab.research.google.com/notebooks/gpu.ipynb
- If colab shows errors in code that was working before try to do `Runtime->Restart Runtime`. This is especially true if you stopped execution while a model was being trained.
  - You can also try `Runtime->Factory Reset Runtime` to get a clean slate (revert installed libraries to colab defaults).

# Answer
Write your answer and its code under this section.

In [13]:
# defining the constituents of a model genome so that we can combine them 
# to make a random model genome
function = ["NC","RC"]
filters = ["4","8","16","32","64","128"]
kernels = ["4","5","6","7"]
af = ["relu","sigmoid"]
nc_layers = []
rc_layers =[]

for i in range(len(function)):
  for j in range(len(filters)):
    for k in range(len(kernels)):
      for l in range(len(af)):
        str = function[i]+" "+filters[j]+" "+kernels[k]+" "+af[l]
        if function[i]=='NC':
          nc_layers.append(str)
        else:
          rc_layers.append(str)
print(nc_layers) 
print(rc_layers)

['NC 4 4 relu', 'NC 4 4 sigmoid', 'NC 4 5 relu', 'NC 4 5 sigmoid', 'NC 4 6 relu', 'NC 4 6 sigmoid', 'NC 4 7 relu', 'NC 4 7 sigmoid', 'NC 8 4 relu', 'NC 8 4 sigmoid', 'NC 8 5 relu', 'NC 8 5 sigmoid', 'NC 8 6 relu', 'NC 8 6 sigmoid', 'NC 8 7 relu', 'NC 8 7 sigmoid', 'NC 16 4 relu', 'NC 16 4 sigmoid', 'NC 16 5 relu', 'NC 16 5 sigmoid', 'NC 16 6 relu', 'NC 16 6 sigmoid', 'NC 16 7 relu', 'NC 16 7 sigmoid', 'NC 32 4 relu', 'NC 32 4 sigmoid', 'NC 32 5 relu', 'NC 32 5 sigmoid', 'NC 32 6 relu', 'NC 32 6 sigmoid', 'NC 32 7 relu', 'NC 32 7 sigmoid', 'NC 64 4 relu', 'NC 64 4 sigmoid', 'NC 64 5 relu', 'NC 64 5 sigmoid', 'NC 64 6 relu', 'NC 64 6 sigmoid', 'NC 64 7 relu', 'NC 64 7 sigmoid', 'NC 128 4 relu', 'NC 128 4 sigmoid', 'NC 128 5 relu', 'NC 128 5 sigmoid', 'NC 128 6 relu', 'NC 128 6 sigmoid', 'NC 128 7 relu', 'NC 128 7 sigmoid']
['RC 4 4 relu', 'RC 4 4 sigmoid', 'RC 4 5 relu', 'RC 4 5 sigmoid', 'RC 4 6 relu', 'RC 4 6 sigmoid', 'RC 4 7 relu', 'RC 4 7 sigmoid', 'RC 8 4 relu', 'RC 8 4 sigmoid', '

In [ ]:
'''
from random import randint
accuracy = []
states = []
genome = nc_layers[0]+";"+rc_layers[0]+";"+rc_layers[1]+";"+"FL relu;"
r= train_model(genome, verbose=2, lr=__fr_lr, lr_decay=__fr_lr_decay)
print(r)
accuracy.append([genome,r[0]*100])
while r[0]*100<75 :
  a = randint(0,len(nc_layers)-1)
  b = randint(0,len(rc_layers)-1)
  c = randint(0,len(rc_layers)-1)
  if r[0]>0.75 and r[0]<0.85:
    genome = nc_layers[a]+";"+nc_layers[b]+";"+rc_layers[b]+";"+rc_layers[c]+";"+"FL relu;"
  else:
    genome = nc_layers[a]+";"+rc_layers[b]+";"+rc_layers[c]+";"+"FL relu;"
  r= train_model(genome, verbose=2, lr=__fr_lr, lr_decay=__fr_lr_decay)
  print(r)
  accuracy.append([genome,r[0]*100])
'''

In [ ]:
'''
for i in range(len(accuracy)):
  print(accuracy[i])
'''

In [11]:
import random

def random_layer(f,l,k,af,layertype):
  while True:
    a = random.randint(0,len(f)-1)
    b = random.randint(0,len(l)-1)
    c = random.randint(0,len(k)-1)
    d = random.randint(0,len(af)-1)
    string = f[a]+" "+l[b]+" "+k[c]+" "+af[d]+";"
    # print(string,string[:2])
    if string[:2]==layertype:
      return string

def random_architecture(k):
  s = []
  a = ''
  for i in range(k-2-1):
    s.append(random_layer(function,filters,kernels,af,"NC"))
  for i in range(2):
    s.append(random_layer(function,filters,kernels,af,"RC"))
  random.shuffle(s)
  s.append("FL relu;")
  for i in range(len(s)):
    a = a + s[i]
  return a
arch = random_architecture(6)
print(arch)

RC 64 4 relu;RC 8 4 relu;NC 32 6 relu;NC 64 7 sigmoid;NC 4 4 relu;FL relu;


In [ ]:
p = 6
layers = 6
population = []
accuracy = []
history = []

# initialize population
for i in range(p):
  x = random_architecture(layers)
  if x not in population:
    population.append(x)
    r = train_model(x, verbose=2, lr=__fr_lr, lr_decay=__fr_lr_decay)
    accuracy.append(r)

Training genome: RC 8 4 relu;NC 32 6 sigmoid;NC 128 7 relu;RC 32 6 relu;NC 16 6 relu;FL relu;
Model params: 377890
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer_0_RC (Conv2D)         (None, 13, 13, 8)         136       
                                                                 
 Layer_1_NC (Conv2D)         (None, 13, 13, 32)        9248      
                                                                 
 Layer_2_NC (Conv2D)         (None, 13, 13, 128)       200832    
                                                                 
 Layer_3_RC (Conv2D)         (None, 4, 4, 32)          147488    
                                                                 
 Layer_4_NC (Conv2D)         (None, 4, 4, 16)          18448     
                                                                 
 global_average_pooling2d_20  (None, 16)               0         
  (G

Training:   0%|           0/10 ETA: ?s,  ?epochs/s

Epoch 1/10


0/469           ETA: ?s - 

Epoch 2/10


...

Epoch 2/10


0/469           ETA: ?s - 

Epoch 3/10


0/469           ETA: ?s - 

Epoch 00003: early stopping
Best test accuracy: 0.10000000149011612


In [ ]:
print(population)
print(accuracy)

['RC 8 4 relu;NC 32 6 sigmoid;NC 128 7 relu;RC 32 6 relu;NC 16 6 relu;FL relu;', 'NC 128 7 sigmoid;RC 32 5 relu;RC 16 4 relu;NC 16 7 relu;NC 16 7 sigmoid;FL relu;', 'NC 32 6 sigmoid;RC 64 5 relu;NC 128 4 sigmoid;NC 4 5 relu;RC 8 6 sigmoid;FL relu;', 'RC 4 6 sigmoid;NC 32 4 relu;NC 4 5 relu;RC 16 5 relu;NC 32 5 sigmoid;FL relu;', 'RC 16 6 relu;NC 16 5 sigmoid;NC 16 6 relu;NC 4 6 relu;RC 64 7 relu;FL relu;', 'RC 128 5 sigmoid;NC 128 5 sigmoid;NC 4 6 sigmoid;NC 8 4 sigmoid;RC 16 7 relu;FL relu;']
[(0.10000000149011612, 0.09863333404064178, 377890), (0.10000000149011612, 0.10001666843891144, 143898), (0.41530001163482666, 0.4171833395957947, 198838), (0.10000000149011612, 0.10014999657869339, 22642), (0.48190000653266907, 0.4827499985694885, 35966), (0.10000000149011612, 0.10071666538715363, 440038)]


In [ ]:
max_acc = max(accuracy)
idx = accuracy.index(max_acc)

print(f"{max_acc},\n{population[idx]}")

(0.48190000653266907, 0.4827499985694885, 35966),
RC 16 6 relu;NC 16 5 sigmoid;NC 16 6 relu;NC 4 6 relu;RC 64 7 relu;FL relu;


In [14]:
arch1 = random_architecture(6)
print(arch1)

arch2 = random_architecture(6)
print(arch2)

RC 64 4 relu;RC 32 6 sigmoid;NC 16 6 sigmoid;NC 16 4 relu;NC 16 6 sigmoid;FL relu;
RC 8 4 sigmoid;RC 64 5 relu;NC 4 4 sigmoid;NC 128 5 relu;NC 64 4 relu;FL relu;


In [ ]:
  la1 = []
  la2 = []
  la1 = arch1.split(';')
  la2 = arch2.split(';')

In [ ]:
la1, la2

(['NC 4 7 sigmoid', 'RC 64 4 relu', 'RC 16 6 relu', 'FL relu', ''],
 ['NC 4 5 relu',
  'NC 64 7 sigmoid',
  'RC 8 7 relu',
  'NC 128 7 sigmoid',
  'RC 32 4 sigmoid',
  'FL relu',
  ''])

In [ ]:
la1_copy = la1[:]

la1_copy.pop()
choice1 = random.choice(la1_copy)
# choice1
la1_copy.remove(choice1)
choice2 = random.choice(la1_copy)

index1, index2 = la1.index(choice1), la1.index(choice2)
la1[index1], la1[index2] = choice2, choice1

print(f"{choice1}\n{choice2}\n{la1}")

NC 128 7 sigmoid
RC 128 6 relu
['RC 128 6 relu', 'NC 128 7 sigmoid', 'RC 4 4 relu', 'FL relu', '']


In [ ]:
s1 = ";".join(la1)
print(s1)

RC 128 6 relu;NC 128 7 sigmoid;RC 4 4 relu;FL relu;


In [32]:
def swap_mutation(model_genome):
  """
  Randomly choose two layers in the model genome and swap them
  """
  # creating list and its copies
  la = []
  la = model_genome.split(';')
  fl = "FL relu"
  la.remove("FL relu")
  la_copy = la[:]  
  # print(f"{la}\n{la_copy}")

  # choosing two indices at random
  choice1 = random.choice(la_copy)
  ### print(choice1)
  la_copy.remove(choice1)
  choice2 = random.choice(la_copy)

  index1, index2 = la.index(choice1), la.index(choice2)
  # swapping two layers in the genome
  la[index1], la[index2] = choice2, choice1  
  print(f"Swapping :\n{choice1}\n{choice2}")
  # print(la)
  
  swapped_genome = ";".join(la) + fl + ";"
  # print(swapped_genome)

  return swapped_genome

def inversion_mutation(model_genome):
  """
  Select a sequence of layers at random and invert them
  """
  la = []
  la = model_genome.split(';')
  fl = "FL relu"
  la.remove("FL relu")
  # main_str = ";".join(la)

  idx1 = random.randint(0, len(la)-2)
  idx2 = random.randint(idx1+1, len(la)-1)
  # substr = get_substring(main_str, substr_len)
  sa = la[idx1:idx2]
  print(f"{la}\n{fl}\n{idx1}\n{idx2}\n{sa}")
  
  sa = sa[::-1]
  la[idx1:idx2] = sa

  inverted_genome = ";".join(la) + fl + ";"

  return inverted_genome



In [ ]:
i = [4, 5, 7, 8, 9, 1]
j = i[1:4]
j = j[::-1]
i[1:4] = j

print(i)

[4, 8, 7, 5, 9, 1]


In [58]:
r1 = swap_mutation(arch1)
print(f"\n{arch1}\n\n{r1}")

Swapping :
NC 16 4 relu
NC 16 6 sigmoid

RC 64 4 relu;RC 32 6 sigmoid;NC 16 6 sigmoid;NC 16 4 relu;NC 16 6 sigmoid;FL relu;

RC 64 4 relu;RC 32 6 sigmoid;NC 16 4 relu;NC 16 6 sigmoid;NC 16 6 sigmoid;FL relu;


In [ ]:
r2 = inversion_mutation(arch2)
print(f"\n{arch2}\n{r2}")

['NC 4 4 relu', 'RC 8 5 sigmoid', 'NC 16 7 sigmoid', 'NC 8 5 sigmoid', 'RC 8 7 sigmoid', '']
FL relu
4
5
['RC 8 7 sigmoid']

NC 4 4 relu;RC 8 5 sigmoid;NC 16 7 sigmoid;NC 8 5 sigmoid;RC 8 7 sigmoid;FL relu;
NC 4 4 relu;RC 8 5 sigmoid;NC 16 7 sigmoid;NC 8 5 sigmoid;RC 8 7 sigmoid;FL relu;


In [59]:
while True:
  samples = []
  for i in range(4):
    a = random.randint(0,len(population)-1)
    if population[a] not in samples:
      samples.append([population[a],accuracy[a]])
  max1,max2 = 0,0
  j,k = 0,0
  for i in range(len(samples)):
    if samples[i][1][0]>max1:
      max2 = max1
      max1 = samples[i][1][0]
      k = j
      j = i
  print(samples[j])
  print(samples[k])
  
  ge = inversion_mutation(samples[j][0])
  print(ge)
  if ge not in population:
    r = train_model(ge, verbose=2, lr=__fr_lr, lr_decay=__fr_lr_decay)
    population.append(ge)
    accuracy.append(r)

  top = []
  for i, v in enumerate(accuracy):
    if v[0] > 0.75:
      top.append((population[i], accuracy[i]))

  if max1>0.90:
    break

  ge = inversion_mutation(samples[j][0])
  print(ge)
  if ge not in population:
    r = train_model(ge, verbose=2, lr=__fr_lr, lr_decay=__fr_lr_decay)
    population.append(ge)
    accuracy.append(r)





NameError: ignored

In [ ]:
print(population)
print(accuracy)

['RC 8 4 relu;NC 32 6 sigmoid;NC 128 7 relu;RC 32 6 relu;NC 16 6 relu;FL relu;', 'NC 128 7 sigmoid;RC 32 5 relu;RC 16 4 relu;NC 16 7 relu;NC 16 7 sigmoid;FL relu;', 'NC 32 6 sigmoid;RC 64 5 relu;NC 128 4 sigmoid;NC 4 5 relu;RC 8 6 sigmoid;FL relu;', 'RC 4 6 sigmoid;NC 32 4 relu;NC 4 5 relu;RC 16 5 relu;NC 32 5 sigmoid;FL relu;', 'RC 16 6 relu;NC 16 5 sigmoid;NC 16 6 relu;NC 4 6 relu;RC 64 7 relu;FL relu;', 'RC 128 5 sigmoid;NC 128 5 sigmoid;NC 4 6 sigmoid;NC 8 4 sigmoid;RC 16 7 relu;FL relu;', 'RC 16 6 relu;NC 16 5 sigmoid;NC 16 6 relu;RC 64 7 relu;NC 4 6 relu;FL relu;', 'RC 64 5 relu;NC 32 6 sigmoid;NC 128 4 sigmoid;NC 4 5 relu;RC 8 6 sigmoid;FL relu;', 'NC 32 6 sigmoid;RC 64 5 relu;NC 4 5 relu;NC 128 4 sigmoid;RC 8 6 sigmoid;FL relu;', 'NC 16 6 relu;NC 16 5 sigmoid;RC 16 6 relu;RC 64 7 relu;NC 4 6 relu;FL relu;', 'RC 64 5 relu;NC 128 4 sigmoid;NC 32 6 sigmoid;NC 4 5 relu;RC 8 6 sigmoid;FL relu;', 'NC 16 6 relu;NC 16 5 sigmoid;NC 4 6 relu;RC 64 7 relu;RC 16 6 relu;FL relu;', 'RC 16 6 

In [ ]:
print(samples[j])

['NC 16 6 relu;NC 16 5 sigmoid;RC 64 7 relu;NC 4 6 relu;RC 16 6 relu;FL relu;', (0.598800003528595, 0.60548335313797, 70526)]


In [ ]:
top

[('NC 16 6 relu;NC 16 5 sigmoid;NC 4 6 relu;RC 64 7 relu;RC 16 6 relu;FL relu;',
  (0.8227999806404114, 0.8281833529472351, 60542))]

## References
- Weng, L. (2020, August 6). Neural Architecture Search. Lil'Log. Retrieved November 17, 2021, from https://lilianweng.github.io/lil-log/2020/08/06/neural-architecture-search.html#evolutionary-algorithms. 
- Neural architecture search based on Evolutionary Algorithms with Fitness Approximation. (n.d.). Retrieved November 17, 2021, from https://cis.ieee.org/images/files/Documents/summer-school-reports/ChaoPan_Report_1.pdf. 